<a href="https://colab.research.google.com/github/ram130849/Deep_Learning_Systems_Assignments/blob/main/TensorFlow/Sushant/DLS_Assignment4_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import timeit
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten
from keras.models import Model
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.2185214090000045
GPU (s):
0.040366400999999996
GPU speedup over CPU: 79x


### Loading the MNIST Dataset

In [ ]:
mnist = tf.keras.datasets.mnist

In [ ]:
(train_images, train_labels) , (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [ ]:
#Lets look at the training data
print("Training Images Shape: ",train_images.shape)
print("Training Labels: ",train_labels)


#Lets look at the testing data
print("Testing Images Shape: ",test_images.shape)
print("Testing Labels: ",test_labels)

Training Images Shape:  (60000, 28, 28)
Training Labels:  [5 0 4 ... 5 6 8]
Testing Images Shape:  (10000, 28, 28)
Testing Labels:  [7 2 1 ... 4 5 6]


### Normalizing the Images

In [ ]:
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images/=255
test_images/=255

### Loading the Model from Part 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_path = '/content/drive/MyDrive/DLS_Assignments/Models/assign4_part1_baseline.h5'

In [ ]:
baseline = keras.models.load_model(model_path)

In [ ]:
evaluation = baseline.evaluate(test_images , test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.1257 - sparse_categorical_accuracy: 0.9856


### Performing Low Rank Approximation on the above model for D=20

In [ ]:
s_1 , U_1, V_1 = tf.linalg.svd(baseline.get_layer(index=1).get_weights()[0])
s_2 , U_2, V_2 = tf.linalg.svd(baseline.get_layer(index=2).get_weights()[0])
s_3 , U_3, V_3 = tf.linalg.svd(baseline.get_layer(index=3).get_weights()[0])
s_4 , U_4, V_4 = tf.linalg.svd(baseline.get_layer(index=4).get_weights()[0])
s_5 , U_5, V_5 = tf.linalg.svd(baseline.get_layer(index=5).get_weights()[0])



S_1 = tf.linalg.diag(s_1)
S_2 = tf.linalg.diag(s_2)
S_3 = tf.linalg.diag(s_3)
S_4 = tf.linalg.diag(s_4)
S_5 = tf.linalg.diag(s_5)



V_1T = tf.transpose(V_1)
V_2T = tf.transpose(V_2)
V_3T = tf.transpose(V_3)
V_4T = tf.transpose(V_4)
V_5T = tf.transpose(V_5)


bias1 = baseline.get_layer(index=1).get_weights()[1]
bias2 = baseline.get_layer(index=2).get_weights()[1]
bias3 = baseline.get_layer(index=3).get_weights()[1]
bias4 = baseline.get_layer(index=4).get_weights()[1]
bias5 = baseline.get_layer(index=5).get_weights()[1]

In [ ]:
r=20
W1 = tf.matmul(U_1[:, :r], tf.matmul(S_1[:r, :r], V_1T[:r, :]))
W2 = tf.matmul(U_2[:, :r], tf.matmul(S_2[:r, :r], V_2T[:r, :]))
W3 = tf.matmul(U_3[:, :r], tf.matmul(S_3[:r, :r], V_3T[:r, :]))
W4 = tf.matmul(U_4[:, :r], tf.matmul(S_4[:r, :r], V_4T[:r, :]))
W5 = tf.matmul(U_5[:, :r], tf.matmul(S_5[:r, :r], V_5T[:r, :]))


# baseline.get_layer(index=1).set_weights([W1,bias1])
# baseline.get_layer(index=2).set_weights([W2,bias2])
# baseline.get_layer(index=3).set_weights([W3,bias3])
# baseline.get_layer(index=4).set_weights([W4,bias4])
# baseline.get_layer(index=5).set_weights([W5,bias5])

In [ ]:
new_model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(1024, activation='relu', name='first_layer',),
    layers.Dense(1024, activation='relu', name='second_layer'),
    layers.Dense(1024, activation='relu', name='third_layer',),
    layers.Dense(1024, activation='relu', name='fourth_layer',),
    layers.Dense(1024, activation='relu', name="fifth_layser",),
    layers.Dense(10, name='output' ,activation='softmax')
])
print(new_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 first_layer (Dense)         (None, 1024)              803840    
                                                                 
 second_layer (Dense)        (None, 1024)              1049600   
                                                                 
 third_layer (Dense)         (None, 1024)              1049600   
                                                                 
 fourth_layer (Dense)        (None, 1024)              1049600   
                                                                 
 fifth_layser (Dense)        (None, 1024)              1049600   
                                                                 
 output (Dense)              (None, 10)               

In [ ]:
new_model.get_layer(index=1).set_weights([W1,bias1])
new_model.get_layer(index=2).set_weights([W2,bias2])
new_model.get_layer(index=3).set_weights([W3,bias3])
new_model.get_layer(index=4).set_weights([W4,bias4])
new_model.get_layer(index=5).set_weights([W5,bias5])

In [ ]:
new_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['acc'],
)

In [ ]:
history_new = new_model.fit(train_images , train_labels ,epochs = 30, batch_size = 256)

Epoch 1/30
235/235 [==============================] - 2s 6ms/step - loss: 0.2264 - acc: 0.9423
Epoch 2/30
235/235 [==============================] - 1s 5ms/step - loss: 0.0392 - acc: 0.9889
Epoch 3/30
235/235 [==============================] - 1s 5ms/step - loss: 0.0239 - acc: 0.9929
Epoch 4/30
235/235 [==============================] - 1s 5ms/step - loss: 0.0206 - acc: 0.9940
Epoch 5/30
235/235 [==============================] - 1s 5ms/step - loss: 0.0157 - acc: 0.9954
Epoch 6/30
235/235 [==============================] - 1s 5ms/step - loss: 0.0144 - acc: 0.9956
Epoch 7/30
235/235 [==============================] - 1s 5ms/step - loss: 0.0126 - acc: 0.9965
Epoch 8/30
235/235 [==============================] - 1s 5ms/step - loss: 0.0115 - acc: 0.9969
Epoch 9/30
235/235 [==============================] - 1s 5ms/step - loss: 0.0087 - acc: 0.9975
Epoch 10/30
235/235 [==============================] - 1s 5ms/step - loss: 0.0071 - acc: 0.9982
Epoch 11/30
235/235 [============================

In [ ]:
print("The average training accuracy is :",np.mean(history_new.history['acc']))

The average training accuracy is : 0.9954722205797831


In [ ]:
evaluation = new_model.evaluate(test_images , test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.1218 - acc: 0.9843


In [ ]:
print("The test accuracy is :",evaluation[1])

The test accuracy is : 0.9843000173568726
